In [0]:
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'intro-to-pytorch'))
	print(os.getcwd())
except:
	pass

 # Inference and Validation

 Now that you have a trained network, you can use it for making predictions. This is typically called **inference**, a term borrowed from statistics. However, neural networks have a tendency to perform *too well* on the training data and aren't able to generalize to data that hasn't been seen before. This is called **overfitting** and it impairs inference performance. To test for overfitting while training, we measure the performance on data not in the training set called the **validation** set. We avoid overfitting through regularization such as dropout while monitoring the validation performance during training. In this notebook, I'll show you how to do this in PyTorch.

 As usual, let's start by loading the dataset through torchvision. You'll learn more about torchvision and loading data in a later part. This time we'll be taking advantage of the test set which you can get by setting `train=False` here:

 ```python
 testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
 ```

 The test set contains images just like the training set. Typically you'll see 10-20% of the original dataset held out for testing and validation with the rest being used for training.

In [4]:
from tqdm import tqdm
import torch
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)


0it [00:00, ?it/s]

26427392it [00:05, 5279637.23it/s]                              


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 36474.57it/s]                           
0it [00:00, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw


4423680it [00:02, 1505831.08it/s]                            
0it [00:00, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw


8192it [00:00, 13473.97it/s]            

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw
Processing...
Done!


 Here I'll create a model like normal, using the same one from my solution for part 4.

In [0]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x


 The goal of validation is to measure the model's performance on data that isn't part of the training set. Performance here is up to the developer to define though. Typically this is just accuracy, the percentage of classes the network predicted correctly. Other options are [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall#Definition_(classification_context)) and top-5 error rate. We'll focus on accuracy here. First I'll do a forward pass with one batch from the test set.

In [6]:
model = Classifier()

images, labels = next(iter(testloader))
# Get the class probabilities
ps = torch.exp(model(images))
# Make sure the shape is appropriate, we should get 10 class probabilities for 64 examples
print(ps.shape)


torch.Size([64, 10])


 With the probabilities, we can get the most likely class using the `ps.topk` method. This returns the $k$ highest values. Since we just want the most likely class, we can use `ps.topk(1)`. This returns a tuple of the top-$k$ values and the top-$k$ indices. If the highest value is the fifth element, we'll get back 4 as the index.

In [7]:
top_p, top_class = ps.topk(1, dim=1)
# Look at the most likely classes for the first 10 examples
print(top_class[:10,:])


tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])


 Now we can check if the predicted classes match the labels. This is simple to do by equating `top_class` and `labels`, but we have to be careful of the shapes. Here `top_class` is a 2D tensor with shape `(64, 1)` while `labels` is 1D with shape `(64)`. To get the equality to work out the way we want, `top_class` and `labels` must have the same shape.

 If we do

 ```python
 equals = top_class == labels
 ```

 `equals` will have shape `(64, 64)`, try it yourself. What it's doing is comparing the one element in each row of `top_class` with each element in `labels` which returns 64 True/False boolean values for each row.

In [0]:
equals = top_class == labels.view(*top_class.shape)


 Now we need to calculate the percentage of correct predictions. `equals` has binary values, either 0 or 1. This means that if we just sum up all the values and divide by the number of values, we get the percentage of correct predictions. This is the same operation as taking the mean, so we can get the accuracy with a call to `torch.mean`. If only it was that simple. If you try `torch.mean(equals)`, you'll get an error

 ```
 RuntimeError: mean is not implemented for type torch.ByteTensor
 ```

 This happens because `equals` has type `torch.ByteTensor` but `torch.mean` isn't implemented for tensors with that type. So we'll need to convert `equals` to a float tensor. Note that when we take `torch.mean` it returns a scalar tensor, to get the actual value as a float we'll need to do `accuracy.item()`.

In [9]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')


Accuracy: 14.0625%


 The network is untrained so it's making random guesses and we should see an accuracy around 10%. Now let's train our network and include our validation pass so we can measure how well the network is performing on the test set. Since we're not updating our parameters in the validation pass, we can speed up our code by turning off gradients using `torch.no_grad()`:

 ```python
 # turn off gradients
 with torch.no_grad():
     # validation pass here
     for images, labels in testloader:
         ...
 ```

 >**Exercise:** Implement the validation loop below and print out the total accuracy after the loop. You can largely copy and paste the code from above, but I suggest typing it in because writing it out yourself is essential for building the skill. In general you'll always learn more by typing it rather than copy-pasting. You should be able to get an accuracy above 80%.

In [0]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 100
steps = 0

train_losses, test_losses = [], []
for e in tqdm(range(epochs)):
    running_loss = 0
    for images, labels in trainloader:
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        ## Implement the validation pass and print out the validation accuracy
        with torch.no_grad():
          test_loss = 0
          accuracy = 0
        
          for images, labels in testloader:
            outputs = model.forward(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predictions = outputs.topk(1)
            equals = (labels == predictions.view(*labels.shape))
            accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        running_loss = running_loss / len(trainloader)
        test_loss = test_loss / len(testloader)
        train_losses.append(running_loss)
        test_losses.append(test_loss)
        accuracy = accuracy / len(testloader)
        print(f'Train loss: {running_loss}')
        print(f'Test loss: {test_loss}')
        print(f'Accuracy: {accuracy.item()*100}%')
print('Train losses')
print(train_losses)
print('Test losses')
print(test_losses)


  1%|          | 1/100 [00:10<16:44, 10.14s/it]

Train loss: 0.5203725083518639
Test loss: 0.44095899890752355
Accuracy: 84.7034215927124%


  2%|▏         | 2/100 [00:20<16:45, 10.26s/it]

Train loss: 0.39387606552986704
Test loss: 0.4678008576297456
Accuracy: 83.77786874771118%


  3%|▎         | 3/100 [00:31<16:41, 10.33s/it]

...

Train loss: 0.11222840438652505
Test loss: 0.7658912431282602
Accuracy: 88.59474658966064%


 93%|█████████▎| 93/100 [19:35<01:54, 16.37s/it]

Train loss: 0.1247593795122114
Test loss: 0.7342923151649488
Accuracy: 88.51512670516968%


 94%|█████████▍| 94/100 [19:51<01:38, 16.41s/it]

Train loss: 0.12249455943669893
Test loss: 0.7067912214548345
Accuracy: 88.86345624923706%


 ## Overfitting

 If we look at the training and validation losses as we train the network, we can see a phenomenon known as overfitting.

 <img src='https://github.com/dinaldoap/deep-learning-v2-pytorch/blob/e5737c87f3c6fe345f8a3a24494d199084456386/intro-to-pytorch/assets/overfitting.png?raw=1' width=450px>

 The network learns the training set better and better, resulting in lower training losses. However, it starts having problems generalizing to data outside the training set leading to the validation loss increasing. The ultimate goal of any deep learning model is to make predictions on new data, so we should strive to get the lowest validation loss possible. One option is to use the version of the model with the lowest validation loss, here the one around 8-10 training epochs. This strategy is called *early-stopping*. In practice, you'd save the model frequently as you're training then later choose the model with the lowest validation loss.

 The most common method to reduce overfitting (outside of early-stopping) is *dropout*, where we randomly drop input units. This forces the network to share information between weights, increasing it's ability to generalize to new data. Adding dropout in PyTorch is straightforward using the [`nn.Dropout`](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout) module.

 ```python
 class Classifier(nn.Module):
     def __init__(self):
         super().__init__()
         self.fc1 = nn.Linear(784, 256)
         self.fc2 = nn.Linear(256, 128)
         self.fc3 = nn.Linear(128, 64)
         self.fc4 = nn.Linear(64, 10)

         # Dropout module with 0.2 drop probability
         self.dropout = nn.Dropout(p=0.2)

     def forward(self, x):
         # make sure input tensor is flattened
         x = x.view(x.shape[0], -1)

         # Now with dropout
         x = self.dropout(F.relu(self.fc1(x)))
         x = self.dropout(F.relu(self.fc2(x)))
         x = self.dropout(F.relu(self.fc3(x)))

         # output so no dropout here
         x = F.log_softmax(self.fc4(x), dim=1)

         return x
 ```

 During training we want to use dropout to prevent overfitting, but during inference we want to use the entire network. So, we need to turn off dropout during validation, testing, and whenever we're using the network to make predictions. To do this, you use `model.eval()`. This sets the model to evaluation mode where the dropout probability is 0. You can turn dropout back on by setting the model to train mode with `model.train()`. In general, the pattern for the validation loop will look like this, where you turn off gradients, set the model to evaluation mode, calculate the validation loss and metric, then set the model back to train mode.

 ```python
 # turn off gradients
 with torch.no_grad():

     # set model to evaluation mode
     model.eval()

     # validation pass here
     for images, labels in testloader:
         ...

 # set model back to train mode
 model.train()
 ```

 > **Exercise:** Add dropout to your model and train it on Fashion-MNIST again. See if you can get a lower validation loss or higher accuracy.

In [0]:
## Define your model with dropout added
class ClassifierDropout(nn.Module):
    def __init__(self):
        super().__init__()
        input_size = 28 * 28
        fc1_size = 256
        fc2_size = 128
        fc3_size = 64
        output_size = 10
        dropout = 0.2
        self.model = nn.Sequential(nn.Linear(input_size, fc1_size),
                                    nn.ReLU(),
                                    nn.Dropout(p=dropout),
                                    nn.Linear(fc1_size, fc2_size),
                                    nn.ReLU(),
                                    nn.Dropout(p=dropout),
                                    nn.Linear(fc2_size, fc3_size),
                                    nn.ReLU(),
                                    nn.Dropout(p=dropout),
                                    nn.Linear(fc3_size, output_size),
                                    nn.LogSoftmax(dim=1))

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        return self.model.forward(x)  


In [0]:
## Train your model with dropout, and monitor the training progress with the validation loss and accuracy
model = ClassifierDropout()
optimizer = optim.Adam(model.parameters(), lr=0.003)
criterion = nn.NLLLoss()
epochs = 100

train_losses, val_losses = [], []
for epoch in tqdm(range(epochs)):
    model.train()
    train_loss = 0
    for images, labels in trainloader:
        outputs = model.forward(images)
        loss = criterion(outputs, labels)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    else:
        with torch.no_grad():
            model.eval()
            val_loss = 0
            accuracy = 0
            for images, labels in testloader:
                outputs = model.forward(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predictions = outputs.topk(1)
                equals = (labels == predictions.view(*labels.shape))
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        train_loss = train_loss / len(trainloader)
        val_loss = val_loss / len(testloader)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        accuracy = accuracy / len(testloader)
        print('Training loss: {}'.format(train_loss))
        print('Validation loss: {}'.format(val_loss))
        print('Accuracy: {}%'.format(accuracy*100))
print('Train losses')
print(train_losses)
print('Validation losses')
print(val_losses)        






  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:10<17:55, 10.87s/it]

Training loss: 0.6034170603478896
Validation loss: 0.49014643783781936
Accuracy: 82.18550872802734%




  2%|▏         | 2/100 [00:22<18:15, 11.17s/it]

Training loss: 0.48117305770484625
Validation loss: 0.44821617386902973
Accuracy: 83.25039672851562%




  3%|▎         | 3/100 [00:34<18:14, 11.28s/it]

Training loss: 0.4466402783735729
Validation loss: 0.406605645065095
Accuracy: 85.19108581542969%




  4%|▍         | 4/100 [00:45<18:09, 11.35s/it]

Training loss: 0.4307492532963946
Validation loss: 0.4340949402113629
Accuracy: 84.58399963378906%


 ## Inference

 Now that the model is trained, we can use it for inference. We've done this before, but now we need to remember to set the model in inference mode with `model.eval()`. You'll also want to turn off autograd with the `torch.no_grad()` context.

In [0]:
# Import helper module (should be in the repo)
import helper

# Test out your network!

model.eval()

dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[0]
# Convert 2D image to 1D vector
img = img.view(1, 784)

# Calculate the class probabilities (softmax) for img
with torch.no_grad():
    output = model.forward(img)

ps = torch.exp(output)

# Plot the image and probabilities
helper.view_classify(img.view(1, 28, 28), ps, version='Fashion')


 ## Next Up!

 In the next part, I'll show you how to save your trained models. In general, you won't want to train a model everytime you need it. Instead, you'll train once, save it, then load the model when you want to train more or use if for inference.